### Code réalisé par l'équipe 8.
### Équipe : Anas Bouhemhem 1953369
###          Bouh Abdillahi 1940646

#### Lien du GitHub : 
https://github.com/konoDioDA253/ELE8702-Lab2

Laboratoire 2


In [27]:
import numpy as np
import pandas as pd 
import yaml
from yaml.loader import SafeLoader
import random
import math
import matplotlib.pyplot as plt
import operator

import os
from pathlib import Path

# Inclusion du fichier de pathloss du 3GPP (Méthodologie 1)
from f3GPP import *



## Section Lecture des fichiers d'entrés

#### Lecture du fichier cas8.yaml

In [28]:
# import pyyaml module
# Open the file and load the file
def open_cas_yaml() :
    with open('cas8.yaml') as f:
        dataCas8 = yaml.load(f, Loader=SafeLoader)
        #print(dataCas8)
        return(dataCas8)




#### Lecture du fichier devices_db.yaml

In [29]:
# import pyyaml module
# Open the file and load the file
def open_devices_yaml() :
    with open('devices_db.yaml') as f:
        dataDevices_db = yaml.load(f, Loader=SafeLoader)
        print(dataDevices_db)
        return(dataDevices_db)

## Création de fichier de coordonnées

### Routine de génération de coordonnées de la structure en grille 

In [30]:
def fill_up_the_lattice(N, lh, lv, nh, nv):
    """Function appelée par get_rectangle_lattice_coords()"""
    
    def get_delta1d(L, n):
        return L/(n + 1)
    
    coords = []
    deltav = get_delta1d(lv, nv)
    deltah = get_delta1d(lh, nh)
    line = 1
    y = deltav
    count = 0
    while count < N:
        if count + nh < N:
            x = deltah
            for  i in range(nh):
                # Fill up the horizontal line
                coords.append((x,y))
                x = x + deltah
                count += 1
            line += 1
        else:
            deltah = get_delta1d(lh, N - count)
            x = deltah
            for i in range(N - count):
                # Fill up the last horizontal line
                coords.append((x,y))
                x = x + deltah
                count += 1
            line += 1
        y = y +deltav
    return coords



def get_rectangle_lattice_coords(lh, lv, N, Np, nh, nv):
    """Function appelee par gen_lattice_coords()"""
    
    if Np > N:
        coords = fill_up_the_lattice(N, lh, lv, nh, nv)
    elif Np < N:
        coords = fill_up_the_lattice(N, lh, lv, nh, nv + 1)
    else:
        coords = fill_up_the_lattice(N, lh, lv, nh, nv)
    return coords


# In[170]:


def gen_lattice_coords(terrain_shape: dict, N: int):
    """Génère un ensemble de N coordonnées placées en grille 
       sur un terrain rectangulaire
    
       Args: terrain_shape: dictionary {'rectangle': {'length' : lh,
                                                   'height' : lv}
           lh and lv are given in the case file"""

    shape = list(terrain_shape.keys())[0]
    lh = terrain_shape[shape]['length']
    lv = terrain_shape[shape]['height']
    R = lv / lh    
    nv = round(math.sqrt(N / R))
    nh = round(R * nv)
    Np = nh * nv
    if shape.lower() == 'rectangle':
        coords = get_rectangle_lattice_coords(lh, lv, N, Np, nh, nv)
    else:
        msg = [f"\tImproper shape ({shape}) used in the\n",
                "\tgeneration of lattice coordinates.\n"
                "\tValid values: ['rectangle']"]
        ERROR(''.join(msg), 2)
    return coords

### Routine des coordonnées en grille

In [31]:
shape= {'rectangle': {'length': 200, 'height': 200}}
N = 32
coords = gen_lattice_coords(shape, N)
print(coords)

[(28.571428571428573, 28.571428571428573), (57.142857142857146, 28.571428571428573), (85.71428571428572, 28.571428571428573), (114.28571428571429, 28.571428571428573), (142.85714285714286, 28.571428571428573), (171.42857142857144, 28.571428571428573), (28.571428571428573, 57.142857142857146), (57.142857142857146, 57.142857142857146), (85.71428571428572, 57.142857142857146), (114.28571428571429, 57.142857142857146), (142.85714285714286, 57.142857142857146), (171.42857142857144, 57.142857142857146), (28.571428571428573, 85.71428571428572), (57.142857142857146, 85.71428571428572), (85.71428571428572, 85.71428571428572), (114.28571428571429, 85.71428571428572), (142.85714285714286, 85.71428571428572), (171.42857142857144, 85.71428571428572), (28.571428571428573, 114.28571428571429), (57.142857142857146, 114.28571428571429), (85.71428571428572, 114.28571428571429), (114.28571428571429, 114.28571428571429), (142.85714285714286, 114.28571428571429), (171.42857142857144, 114.28571428571429), (

### Calcul de Pathloss

In [32]:
def calcul_pathloss(distance_ue_antenne):
    """Calculate the pathloss (substitution for the 3gpp formulas)"""
    if  (distance_ue_antenne == 0) :
        pathloss=500*random.random()
    else : 
        pathloss = distance_ue_antenne
    return pathloss


### Section Lab 2

In [33]:
# #pathCoord = os.path.abspath("") + '\\' + "test" 

# # Lecture de fichier coordonées si existant:

# isExistingCoord = os.path.isfile("coords_lab2_eq8.txt")
# print(isExistingCoord)
# #if (isExistingCoord == false):
#     #création du fichier.


# # Define Data : 

def WriteFichier(dataCas8):

    if 'read' in dataCas8['ETUDE_IMPORTANT']['COORD_FILES']: # Si le ficher est en mode read

        # On change en mode write

        dataCas8['ETUDE_IMPORTANT']['COORD_FILES'] = "write: data/sortie_case8.txt"
        with open(f'cas8.yaml', 'w') as file:   
                yaml.dump(dataCas8,file,sort_keys=False) #Recriture du fichier yaml. 
    
    #On perd les commentaires malheureusement.
    
    print(dataCas8)
    return dataCas8

def ReadFichier(dataCas8):
     
    if 'write' in dataCas8['ETUDE_IMPORTANT']['COORD_FILES']: # Si le ficher est en mode read

        # On change en mode write

        dataCas8['ETUDE_IMPORTANT']['COORD_FILES'] = "write: data/sortie_case8.txt"
        with open(f'cas8.yaml', 'w') as file:   
                yaml.dump(dataCas8,file,sort_keys=False) #Recriture du fichier yaml. 
    
    #On perd les commentaires malheureusement.
    
    print(dataCas8)
    return dataCas8


Test

In [34]:
dataCas8 = open_cas_yaml()
WriteFichier(dataCas8)

# if 'write' in dataCas8['ETUDE_IMPORTANT']['COORD_FILES']:
#     print("Succes!")
# else :
#     print("Failure!")
#     dataCas8['ETUDE_IMPORTANT']['COORD_FILES'] = "write: data/sortie_case8.txt"


# if 'read' in dataCas8['ETUDE_IMPORTANT']['COORD_FILES']:
#     print("good!")
# else :
#     print("bad!")

# yaml_output = yaml.dump(data, sort_keys=False) 


    

{'ETUDE_IMPORTANT': {'SCENARIO': 'Uma', 'ANT_COORD_GEN': 'g', 'UE_COORD_GEN': 'a', 'COORD_FILES': 'write: data/sortie_case8.txt', 'DEVICES': {'Antenna1': {'number': 25}, 'UE1-App1': {'number': 100}, 'UE2-App2': {'number': 200}}, 'VISIBILITY': {'read': 'visibility1.txt'}, 'GEOMETRY': {'Surface': {'rectangle': {'length': 1000, 'height': 500}}}}}


{'ETUDE_IMPORTANT': {'SCENARIO': 'Uma',
  'ANT_COORD_GEN': 'g',
  'UE_COORD_GEN': 'a',
  'COORD_FILES': 'write: data/sortie_case8.txt',
  'DEVICES': {'Antenna1': {'number': 25},
   'UE1-App1': {'number': 100},
   'UE2-App2': {'number': 200}},
  'VISIBILITY': {'read': 'visibility1.txt'},
  'GEOMETRY': {'Surface': {'rectangle': {'length': 1000, 'height': 500}}}}}

## Section Antennes/UE


### Objet Antennes et UE (User Equipement)

In [35]:

class Antenna:
    
    def __init__(self, id):
        self.id = id #id de l'antenne (int)
        self.group = None # group défini dans la base de données (str)
        self.coords = None # tuple contenant les coordonnées (x,y) 
        self.assoc_ues = [] # liste avec les id des UEs associés à l'antenne
        self.scenario = None # scénario tel que lu du fichier de cas (str)
        self.gen = None # type de géneration de coordonnées: 'g', 'a', etc. (str)
        
    
class UE:
    
    def __init__(self, id, app_name):
        self.id= id #id de l'UE (int)
        self.group = None # group défini dans la base de données (str)
        self.coords=None # tuple contenant les coordonnées (x,y) 
        self.app=app_name #nom de l'application qui tourne dans le UE (str)
        self.assoc_ant=None #id de l'antenne associée à l'UE (int)
        self.los = True # LoS ou non (bool)
        self.gen = None # type de géneration de coordonnées: 'g', 'a', etc. (str)
       

### Creation d'objets Antennes/UE et determination des coords

In [36]:
def place_ue_and_antennas(dataCas8):

###################################################
#              Section Antenne   
##################################################

    shape = dataCas8['ETUDE_IMPORTANT']['GEOMETRY']['Surface']
    global Nb_antenne
    Nb_antenne = dataCas8['ETUDE_IMPORTANT']['DEVICES']['Antenna1']['number'] 
    
    global listAntenne # facon d'acceder globalement a la liste
    listAntenne = []

    for x in range(N):
        listAntenne.append(Antenna(x))
        


    # Generation des coordonnees
    coords_antennes = gen_lattice_coords(shape, N)
  
    for x in range(N):
        listAntenne[x].coords = ([item[0] for item in coords_antennes][x],
                                [item[1] for item in coords_antennes][x])



    #############################
    # Test a des fin pratiques, a effacer apres
    #############################
    
    coords_antennes_x = [item[0] for item in coords_antennes]
    coords_antennes_y = [item[1] for item in coords_antennes]

    


###################################################
#              Section UE   
##################################################

    global nb_ue
    nb_ue =  \
    dataCas8['ETUDE_IMPORTANT']['DEVICES']['UE1-App1']['number'] + \
    dataCas8['ETUDE_IMPORTANT']['DEVICES']['UE2-App2']['number']


    nb_ue_app_1 = dataCas8['ETUDE_IMPORTANT']['DEVICES']['UE1-App1']['number']
    nb_ue_app_2 = dataCas8['ETUDE_IMPORTANT']['DEVICES']['UE2-App2']['number']
    
    coords_ue_x = []
    coords_ue_y = []
    coords_ue = []

    #*******************************************
    #   Creation des objets UE
    #*******************************************

    global listUE # facon d'acceder globalement a la liste (voir Antenne)
    listUE = []

    #Qte de UE app 1

    for x in range(nb_ue_app_1):
        listUE.append(UE(x, 'UE1-App1'))

    #Qte de UE app 2

    for x in range(nb_ue_app_2):
        listUE.append(UE(nb_ue_app_1 + x, 'UE2-App2'))


    #grid element layout generation and attribution

    for x in range(nb_ue):
        temp = dataCas8['ETUDE_IMPORTANT']['GEOMETRY']['Surface']['rectangle']['length'] \
            *random.random()
        coords_ue_x.append(temp)
        temp2 = dataCas8['ETUDE_IMPORTANT']['GEOMETRY']['Surface']['rectangle']['height'] \
            *random.random()
        coords_ue_y.append(temp2)

    #index = 0
    #while index < nb_ue:
    #    coords_ue.append((coords_ue_x[index], coords_ue_y[index]))
    #    index = index + 1


    # Attribution des coords dans les UE

    for x in range(len(listUE)):
        setattr(listUE[x], "coords", (coords_ue_x[x], coords_ue_y[x]))
    print(coords_ue)


    #calcul de distance 
    distance = []

    return listAntenne, listUE




Section de test d'affichage


In [37]:
def WriteFichierCoordonnees(listAntenne,listUE):
    WriteFichier(dataCas8)

    tableau = []
    alpha = dataCas8['ETUDE_IMPORTANT']['DEVICES']['UE1-App1']['number']
    for x in range(Nb_antenne):
        tableau.append(["antenna",listAntenne[x].id, listAntenne[x].group, listAntenne[x].coords[0], listAntenne[x].coords[1]]) 
    for y in range(dataCas8['ETUDE_IMPORTANT']['DEVICES']['UE1-App1']['number']):
        tableau.append(["ue app1",listUE[y].id, listUE[y].app, listUE[y].coords[0], listUE[y].coords[1], listUE[y].app]) 
    for z in range(dataCas8['ETUDE_IMPORTANT']['DEVICES']['UE1-App1']['number']):
        tableau.append(["ue app1",listUE[z + alpha].id, listUE[z + alpha].app, listUE[z + alpha].coords[0], listUE[z+ alpha].coords[1], listUE[z + alpha].app]) 

    write_file_assoc("coords_lab2_eq8.txt", str(tableau))    
    return tableau




In [38]:
Datacas8 = open_cas_yaml()
listAntenne,listUE = place_ue_and_antennas(Datacas8)
test = WriteFichierCoordonnees(listAntenne,listUE)
print(test)
#f = open("myfile.txt", "x")
# f.write(str(test))
# f = open("myfile.txt", "w")

[]
{'ETUDE_IMPORTANT': {'SCENARIO': 'Uma', 'ANT_COORD_GEN': 'g', 'UE_COORD_GEN': 'a', 'COORD_FILES': 'write: data/sortie_case8.txt', 'DEVICES': {'Antenna1': {'number': 25}, 'UE1-App1': {'number': 100}, 'UE2-App2': {'number': 200}}, 'VISIBILITY': {'read': 'visibility1.txt'}, 'GEOMETRY': {'Surface': {'rectangle': {'length': 1000, 'height': 500}}}}}
[['antenna', 0, None, 200.0, 55.55555555555556], ['antenna', 1, None, 400.0, 55.55555555555556], ['antenna', 2, None, 600.0, 55.55555555555556], ['antenna', 3, None, 800.0, 55.55555555555556], ['antenna', 4, None, 200.0, 111.11111111111111], ['antenna', 5, None, 400.0, 111.11111111111111], ['antenna', 6, None, 600.0, 111.11111111111111], ['antenna', 7, None, 800.0, 111.11111111111111], ['antenna', 8, None, 200.0, 166.66666666666669], ['antenna', 9, None, 400.0, 166.66666666666669], ['antenna', 10, None, 600.0, 166.66666666666669], ['antenna', 11, None, 800.0, 166.66666666666669], ['antenna', 12, None, 200.0, 222.22222222222223], ['antenna', 13

In [39]:
#def calcul_association_ue_antenne(pathloss_computation, coords_ue, coords_antennes, N_ue, N_antenne):

def calcul_association_ue_antenne(pathloss_computation):
     
    index_antennes = 0
    index_ue = 0
    smallestDistance = [0] * nb_ue
    check = 0
    calculPathlossChaqueAntenne = [] * Nb_antenne # nom a changer
    pathlossTotal = [] * nb_ue 

    temporary_value = 0
    for x in range(nb_ue):
        for y in range(Nb_antenne):
            temp = tuple(map(operator.sub, listUE[x].coords, listAntenne[y].coords))
            # print("HERE ARE COORDS OF LISTUE AND LISTANTENNE : ", listUE[x].coords, listAntenne[y].coords)

            if pathloss_computation == "random" :
                check = calcul_pathloss(0) #random pathloss
                calculPathlossChaqueAntenne.append(calcul_pathloss(0))                

            if pathloss_computation == "distance" :
                check = calcul_pathloss(math.sqrt(math.pow(temp[0],2) + math.pow(temp[1],2))) #pathloss from the distance
                print(check)
                calculPathlossChaqueAntenne.append(check)
                
            if y == 0: # Assigner premiere valeur
                path_loss_min = check 
                smallestDistance[x] = (check, y) #antenne commence a 1, et non a 0.
                # print("Here is pathloss min : ", path_loss_min)
                # print("Here is check : ", check)
                # print("Here is y : ", y)
            elif check < path_loss_min: #sinon comparer pour la plus petite
                path_loss_min = check
                smallestDistance[x] = (check, y) #antenne commence a 1, et non a 0.
                temporary_value = y
                # print("NOUS SOMMES DANS LE ELIF !!! \n")
                # print("Here is pathloss min : ", path_loss_min)
                # print("Here is check : ", check)
                # print("Here is y : ", y)
        # path_loss_min = 1000000000000
        #print(calculPathlossChaqueAntenne)
        pathlossTotal.append(calculPathlossChaqueAntenne)
        calculPathlossChaqueAntenne = [] * Nb_antenne # nom a changer
        setattr(listUE[x], "assoc_ant", (temporary_value))

    
    # print(pathlossTotal)

    return smallestDistance, pathlossTotal   



def set_assoc_ues():
    temp = []
    for ue in listUE :
        temp.append(ue.assoc_ant)
        # listAntenne[temp]

        # setattr(listAntenne[temp], "assoc_ues", (temporary_value))
    
    print(temp) 
    id_antenne_ue = []   
    # for ue in listUE:
    for ant_number in range(Nb_antenne) :
        # print(ant_number)
        for z in range(nb_ue) :
            if (temp[z] == ant_number) :
                id_antenne_ue.append(z) #append numero ue
        print("here is list of ue for antenna number " , ant_number, " : ", id_antenne_ue)
        setattr(listAntenne[ant_number], "assoc_ues", (id_antenne_ue))
        id_antenne_ue = []


# set_assoc_ues()
# print(listAntenne[4].__dict__)
    


### Section Test 

In [40]:
# smallestDistance, pathlossTotal = calcul_association_ue_antenne("distance")
# print("Here is Pathloss pour UE 0 : ", pathlossTotal[0])
# print("Here is PathlossTotal : ", pathlossTotal)

### Écriture dans un fichier

In [41]:
def write_file_assoc(destination, to_write):
    
    
    

    # id_ue, numerant, pathloss = calcul_association_ue_antenne(pathloss_computation, coords_ue, coords_antennes, N_ue, N_antenne)

    
    # data = {"ID UE": id_ue, "ID antenne": numerant,
    # "Pathloss entre UE et antenne": pathloss
    # }

    # id_ant, numerue, pathloss = inv_calcul_association_ue_antenne(pathloss_computation, coords_ue, coords_antennes, N_ue, N_antenne)
    # data_inv = {"ID antenne": id_ant, "ID UE": numerue, 
    # "Pathloss entre UE et antenne": pathloss
    # }

    # df = pd.DataFrame(data=data)
    # df_inv = pd.DataFrame(data=data_inv)

    # grp = df.sort_values(by=['ID antenne'])
    # new_df = grp['ID antenne'].tolist()
    # new_df_id_ue = grp['ID UE'].tolist()
    # list_antenne_sorted = list(range(N_antenne + 1))
    # list_antenne_sorted = list_antenne_sorted[1:]
    # buffer = []
    # vector = [0]*N_antenne
    # current_ue_selection = ""

    # #This let's us associate an antenna to a list of specified UEs
    # temp = 0
    # for j in  id_ue :
    #     if temp == 0 and j == 0 :
    #         current_ue_selection = current_ue_selection + str(new_df_id_ue[j-1] ) + " "
    #     if j < 300 and j > 0 :
    #         if new_df[j-1] == new_df[j] and j < 300 and j > 0 :
    #             current_ue_selection = current_ue_selection + str(new_df_id_ue[j-1]) + " "
    #         else :
    #             buffer.append(current_ue_selection)
    #             vector[temp] = buffer 
    #             buffer = []
    #             current_ue_selection = ""
    #             temp = temp + 1

    #     if temp == (N_antenne-1) :
    #         if (j == N_ue):
    #             current_ue_selection = current_ue_selection + " " + str(new_df_id_ue[j-1])
    #             buffer.append(current_ue_selection)
    #             vector[temp] = buffer 

    # #Dataframe to encapsulate the association antenna to list of UEs
    # data_assoc = { "ID antenne": list_antenne_sorted, "List ID UE": vector
    # }
    # df_assoc = pd.DataFrame(data=data_assoc)
    



    # Writing necessary files
    path = os.path.abspath("") + os.sep + destination 
    # path_inv = os.path.abspath("") + os.sep + assoc_lab1_rand_eqn 
    with open(path, 'w') as f:
        # df_string = df.to_string(header=False, index=False)
        f.write(to_write)
    f.close()
    # with open(path_inv, 'w') as f_inv:
    #     # df_string_inv = df_assoc.to_string(header=False, index=False)
    #     f_inv.write("df_string_inv")
    # f_inv.close()

    return




### Fonction lab2_eq8


In [42]:
def lab2_eq8() :
    # N_ue = 300
    # N_antenne = 25
    dataCas8 = open_cas_yaml()
    dataDevices_db = open_devices_yaml()
    listAntenne, listUE = place_ue_and_antennas(dataCas8)


    user_choose_name = input("Voulez-vous choisir le nom des fichiers à ecrire? ")   
    user_choose_pl_computation = input("Comment calculer pathloss? (par defaut proportionnel a distance)")   

    if (user_choose_pl_computation == "random") :
        #pathloss random
        if user_choose_name == "oui" :
            pl_lab1_eqn = input("Quel nom de fichier pour pl_lab2_rand_eqn.txt ")
            assoc_lab1_eqn = input("Quel nom de fichier pour assoc_lab2_rand_eqn.txt ")
        else :
            pl_lab1_eqn = "pl_lab1_rand_eq8.txt"
            assoc_lab1_eqn = "assoc_lab1_rand_eq8.txt" 

        pathloss_computation = "random"
        # write_file_assoc(pathloss_computation, pl_lab1_eqn, assoc_lab1_eqn, coords_ue, coords_antennes, N_ue, N_antenne)    
        smallestDistance, pathlossTotal  = calcul_association_ue_antenne(pathloss_computation)
        set_assoc_ues()
        print(listAntenne[4].__dict__)
    
    else :
        #pathloss proportional to distance
        if user_choose_name == "oui" :
            pl_lab1_eqn = input("Quel nom de fichier pour pl_lab2_dist_eqn.txt ")
            assoc_lab1_eqn = input("Quel nom de fichier pour assoc_lab2_dist_eqn.txt ")
        else :
            pl_lab1_eqn = "pl_lab2_dist_eq8.txt"
            assoc_lab1_eqn ="assoc_lab2_dist_eq8.txt"

        pathloss_computation = "distance"
        # write_file_assoc(pathloss_computation, pl_lab1_eqn, assoc_lab1_eqn, coords_ue, coords_antennes, N_ue, N_antenne)    
        smallestDistance, pathlossTotal  = calcul_association_ue_antenne(pathloss_computation)
        set_assoc_ues()
        # print(listAntenne[4].__dict__)


    return listAntenne, listUE,  pathlossTotal  


### Fonction lab2()

In [43]:
def lab2() :
    listAntenne, listUE,  pathlossTotal   = lab2_eq8()
    string_assoc = ""
    for x in range(nb_ue) :
        print("UE numero ", listUE[x].id, " vaut : ", listUE[x].__dict__)
    
    # file creation : assoc_lab1_dist_eq8.txt 
    for y in range(Nb_antenne) :
        print("Antenne numero ", listAntenne[y].id, " vaut : ", listAntenne[y].__dict__)
        print("Antenne numero ", listAntenne[y].id, " a les ues suivantes : ", str(listAntenne[y].assoc_ues))
        test_str = str(listAntenne[y].assoc_ues)

        rem_char = "["
        rem_char2 = "]"
        rem_char3 = ","
        res = test_str.replace(rem_char, "")
        res2 = str(res.replace(rem_char2, ""))
        listAntenne_treated = str(res2.replace(rem_char3, ""))
        if (listAntenne_treated == "") :
            listAntenne_treated = "NULL"
        string_assoc_temp = str(listAntenne[y].id) + " " + listAntenne_treated + "\n"
        string_assoc = string_assoc + string_assoc_temp
    # print("srting assoc is : ", string_assoc)
    write_file_assoc("assoc_ant_lab2_eq8.txt", string_assoc)

    # file creation : pl_lab1_dist_eq8.txt.txt 
    string_pl = ""
    for x in range(nb_ue) :
        # print("UE numero ", listUE[x].id, " vaut : ", listUE[x].__dict__)
        # print("UE numero ", listUE[x].id, " a les pathloss suivants : ", str(pathlossTotal[x]))
        count = 0
        for pathloss in pathlossTotal[x] :
            # print("UE numero ", listUE[x].id, "et antenne " , listAntenne[count].id, "ont le pathloss suivant : ", pathloss)
            ue_id = str(listUE[x].id)
            antenne_id = str(listAntenne[count].id)
            pathloss_str = str(pathloss)
            count = count + 1
            string_pl = string_pl + ue_id + " " + antenne_id + " " + pathloss_str + "\n"
    # print("Here is string_pl to BE ADDED in the file :::::::::: ", string_pl)
    # print("srting assoc is : ", string_assoc)
    write_file_assoc("pl_lab2_eq8.txt", string_pl)

    # file creation : assoc_ue_lab2_eq8.txt 
    list_ues = []
    string_assoc_ue = ""
    for y in range(Nb_antenne) :
        print("-------------------------------------------------------------------------------------------------")
        print("Antenne numero ", listAntenne[y].id, " vaut : ", listAntenne[y].__dict__)
        print("Antenne numero ", listAntenne[y].id, " a les ues suivantes : ", listAntenne[y].assoc_ues)
        list_ues = listAntenne[y].assoc_ues
        print("Type of UE associees dans antenne 0 : ", listAntenne[y].assoc_ues)
        if (list_ues != []) :
            print("List of UE associees dans antenne 0 : ", list_ues[0])
            for ue in list_ues : 
                ue_id = str(ue)
                antenne_id = str(listAntenne[y].id)
                # print("Current UE-ANT combo is : ", ue_id, " ", antenne_id)
                string_assoc_ue = string_assoc_ue + ue_id + " " + antenne_id + "\n"
    print("UE-Antenna combo list, TO ADD IN FILE ::::: ", string_assoc_ue)
    write_file_assoc("assoc_ue_lab2_eq8.txt", string_assoc_ue)
        
    WriteFichierCoordonnees(listAntenne,listUE)
        # test1 = str(listAntenne[y].assoc_ues)
        # rem_char = "["
        # rem_char2 = "]"
        # res = test1.replace(rem_char, "")
        # res2 = str(res.replace(rem_char2, ""))
        # test1 = res2
        # output = [int(s) for s in test1.split(',')]
        # print(output)


    # return listAntenne, listUE
    

### Fonction Main

In [44]:
lab2()

{'ANTENNAS': {'Antenna1': {'type': 'antenna', 'name': 'Antenna1', 'height': 10, 'scenario': 'UMi', 'frequency': 28, 'gain': 40}, 'Antenna2': {'type': 'antenna', 'name': 'Antenna2', 'height': 35, 'scenario': 'RMa', 'frequency': 0.9, 'gain': 45}, 'Antenna3': {'type': 'antenna', 'name': 'Antenna1', 'height': 10, 'scenario': 'UMi', 'frequency': 28, 'gain': 45}, 'Antenna4': {'type': 'antenna', 'name': 'Antenna2', 'height': 37, 'scenario': 'RMa', 'frequency': 0.9, 'gain': 45}}, 'UES': {'UE1-App1': {'type': 'ue', 'name': 'UE1-App1', 'app': 'app1', 'height': 1.5}, 'UE2-App2': {'type': 'ue', 'name': 'UE2-App2', 'app': 'app2', 'height': 1.5}}}
[]
66.73929598580209
137.57145853645306
336.7457182786643
536.5346692937745
73.29089720230702
140.86634867095003
338.10516228394414
537.3889404688365
111.63250916704602
164.11456266308954
348.43272270735946
543.945893291263
160.38989400618397
200.4911236800309
366.9720197094705
556.0038091819285
212.51084349244883
244.1979344241851
392.56133756676184
573.2

## Traitement de scénario de propagation 3GPP (UMi, UMa, etc.)

Le module python qui fait les calculs de pathloss du 3GPP. Veillez l'appeler ```calcul_pathloss_3gpp_eqn.py```

## Calcul de pathloss basés sur ces scénarios
Faire des calculs de pathloss basés sur ces scénarios


## Association des UEs pour chaque antenne
Pour chaque UE, montrer l'antenne associée

## Montrer la liste des UEs associés pour CHAQUE Antenne
Pour chaque antenne, montrer la liste des UEs qu'y sont associés.

## Fournir les fichiers suivants :


    Le fichier de cas utilisé, appelé ```cas_lab2_eqn.yaml```
    Le fichier de coordonnées, appelé ```coords_lab2_eqn.txt``` (voir format dans section méthodologie)
    Le fichier de visibilité, appelé ```visibility_lab2_eqn.txt``` (voir format dans exemple ci-dessus)
    Un fichier d'association, appelé ```assoc_ant_lab2_eqn.txt``` qui montre pour chaque antenne, la liste des UEs associées (voir format dans livrables Lab1)
    Un fichier d'association, appelé ```assoc_ue_lab2_eqn.txt``` qui montre pour chaque UE, l'antenne associé
    Le fichier de pathloss, appelé ```pl_lab2_eqn.txt```  qui montre le pathloss pour chaque UE et antenne  (voir format dans livrables Lab1)

## Conclusion

Nous observons dans ce laboratoire que les réseaux 5G de communication sont modélisable sous forme de grille où nous plaçons des Antennes et des User Terminal (UE)
La modélisation du pathloss n'a pas pu être achevée pour ce laboratoire, les formules sont présentes dans le fichier f3GPP.py.

Si ces formules étaient implémentées, nous nous attendons à observer une augmentation du pathloss en fonction de la distance UE-Antenne, d'où l'intérêt de chosir la combinaisaon UE-Antenne avec le pathloss le plus faible en premier lieu.

Nous observons que bien souvent, des Antenne se retrouvent sans UE lorsque le nombre d'UE est faible (observable en modifiant les valeurs dans le fichier cas8.yaml), ce qui explique l'analyse de marché de la part des opérateurs de réseau 5G avant de placer une antenne 5G quelque part en ville : il faut déduire si l'antenne sera rentable en terme de traffic, et donc en terme de User Equipment (UE) présents. 
